In [1]:
import requests
import json
import pandas as pd

In [47]:
o = pd.read_csv('data/o.csv')
o

,id,x,y
0,0,116.250477,39.858049
1,1,116.479124,40.022752
2,2,116.421770,39.787453
3,3,116.352454,39.884982
4,4,116.462390,39.922632
...,...,...,...
95,95,116.445243,39.759413
96,96,116.218084,39.972427
97,97,116.481806,39.813219
98,98,116.327319,39.865851


In [48]:
key = '0ecba51612cda8eaf3ec74db79c96ec0'
header = {'User-Agent': "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50"}

In [60]:
def busway(o_x,o_y):
    url = 'https://restapi.amap.com/v3/direction/transit/integrated?origin={},{}&destination=116.409213,39.499307&city=010&output=json&strategy=0&key={}'.format(o_x,o_y,key)
    r = requests.get(url, headers=header)
    r.encoding = 'utf-8'
    # 将请求得到的数据按照'utf-8'编码成字符串
    data = r.text
    content = json.loads(data)
    return content
def driveway(o_x,o_y):
    url = 'https://restapi.amap.com/v3/direction/driving?origin={},{}&destination=116.409213,39.499307&extensions=all&strategy=1&output=json&key={}'.format(o_x,o_y,key)
    r2 = requests.get(url, headers=header)
    r2.encoding = 'utf-8'
    # 将请求得到的数据按照'utf-8'编码成字符串
    data = r2.text
    content = json.loads(data)
    return content

In [53]:
cost_taxi = []
cost_bus = []
time_bus = []

for i in o.values:
    x = i[1]
    y = i[2]
    content1 = busway(x,y)
    if content1['status'] =='1':
        cost_taxi_i = int(content1['route']['taxi_cost'])
        cost_taxi.append(cost_taxi_i)
        cost_bus_i = content1['route']['transits'][0]['cost']
        cost_bus.append(cost_bus_i)
        time_bus_i =  int(content1['route']['transits'][0]['duration'])/60
        time_bus.append(time_bus_i)

        

In [61]:
time_taxi = []
for i in o.values:
    x = i[1]
    y = i[2]
    content2 = driveway(x,y)
    if content2['status'] == '1':
        time_taxi_i = int(content2['route']['paths'][0]['duration'])/60
        time_taxi.append(time_taxi_i)

In [67]:
output = pd.DataFrame()
output['Longitude'] = o.x
output['Latitude'] = o.y
output['time_taxi'] = time_taxi
output['time_bus'] = time_bus
output['cost_taxi'] = cost_taxi
output['cost_bus'] = cost_bus

In [69]:
output.to_csv('outcome.csv')